In [1]:
#!/usr/bin/env python3
#pip install  rioxarray==0.3.1
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
import numpy as np
import geopandas as gpd
import cartopy.crs as ccrs
import rasterio
import os
import matplotlib.colors
scriptsdir = os.getcwd()
from scipy.interpolate import griddata
from functools import reduce
import xarray
import itertools
import argparse
import matplotlib.colors as mcolors


In [2]:
models=["GAM","GBM"]
taxas=["Mammals"]

In [10]:
ds

NameError: name 'ds' is not defined

In [55]:

time=[1146]
years= ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050','2052', '2056', '2080', '2100', '2150', '2200', '2250']
year_indices = {1146:2,35: 9, 65: 12, 85: 13}
selected_year = years[year_indices[time[0]]]
if time[0] == 1146:
    model_names = ['EWEMBI']
    bioscen_model_names = ['EWEMBI']
       
combinations = list(itertools.product(models, model_names))
    # Load necessary data
convcodes = pd.read_csv("/storage/homefs/ch21o450/scripts/BioScenComb/data/IUCN_LUH_converion_table_Carlson.csv")


for taxa in taxas:# Get all possible combinations of models and model_names    
    for model in models :
        for model_name in model_names:
            for bioscen_model_name in bioscen_model_names:

                convcodes = pd.read_csv("/storage/homefs/ch21o450/scripts/BioScenComb/data/IUCN_LUH_converion_table_Carlson.csv")
                dir_habclass = "/storage/homefs/ch21o450/IUCN/Habitat_Classifications/" + taxa + "/"

                dir_species = "/storage/workspaces/wa_climate/climate_trt/data/BioScen15/individual_projections/" + taxa+ "_" + model +"_results_climate/"
                available_file = os.listdir(dir_species)

                results = []
                for i, species_name in enumerate(formatted_names):
                    formatted_species_name = formatted_species_name =  "Crocidura_olivieri"

                    for file_name in available_file:
                        if formatted_species_name in file_name and model + '_dispersal.csv.xz' in file_name:
                            species_file = file_name
                            species_file2 = [x.split(".csv")[0] for x in species_file] 
                            break
                    else:
                        bioscen_species = None
                        continue

                    bioscen_species = pd.read_csv(dir_species + file_name)

                    available_files_iucn = formatted_species_name + ".csv"
                    if available_files_iucn in os.listdir(dir_habclass):
                        IUCN = pd.read_csv(dir_habclass + available_files_iucn)
                    else:
                        continue

                    lon = bioscen_species["x"]
                    lat = bioscen_species["y"]
                    z = bioscen_species[bioscen_model_name + '_' + selected_year]

                    df = pd.DataFrame({"lon": lon, "lat": lat, "vals": z})
                    df = df.fillna(0)
                    convcodes_renamed = convcodes.rename(columns={'IUCN_hab':'result.code'})
                    IUCN['result.code'] = pd.to_numeric(IUCN['result.code'], errors='coerce')
                    Habitats = IUCN.merge(convcodes_renamed, left_on='result.code', right_on='result.code')

                    keys = ['LUH1', 'LUH2', 'LUH3', 'LUH4', 'LUH5', 'LUH6', 'LUH7', 'LUH8','LUH9','LUH10', 'LUH11', 'LUH12']
                    split_cols = Habitats['LUH'].str.split('.', expand=True)
                    for i, key in enumerate(keys):
                        if i < len(split_cols.columns):
                            Habitats[key] = split_cols[i]
                        else:
                            Habitats[key] = pd.Series(dtype='float64')
                    if len(Habitats.columns) > len(keys) + 1:
                        num_missing_cols = len(Habitats.columns) - len(keys) - 1
                        Habitats = Habitats.reindex(columns=list(Habitats.columns) + ['LUH{}'.format(i) for i in range(13, 13 + num_missing_cols)], fill_value=np.nan)
                        Habitats.drop('LUH', axis=1, inplace=True)
                    Habitats_suitable = Habitats[Habitats['result.suitability'] == 'Suitable'].copy()

                    LandUseList = "/storage/workspaces/wa_climate/climate_trt/chari/LUH2/remapped_luh2_historical.nc"

                    #isimip = xr.open_dataarray("/storage/workspaces/wa_climate/climate_trt/data/ISIMIP/ISIMIP3b/InputData/GCM/global/miroc6_r1i1p1f1_w5e5_ssp585_tasmin_global_daily_2071_2080.nc")


                    ncfname = LandUseList
                    da_landuse =  xr.open_dataset(ncfname, decode_times=False)
                    da_landuse = da_landuse.isel(time=time)

                    #prifdf_bin = xr.where(prifdf > 0, 1, 0)
                    df_sdm =df

                    #build an empty np.array 
                    np_empty = np.zeros_like(da_landuse['primf'].values, dtype=float)

                    #isimip_lats = isimip['lat'].values
                    #isimip_lons = isimip['lon'].values

                    lats = da_landuse['lat'].values
                    lons = da_landuse['lon'].values

                    da_empty = xr.DataArray(np_empty, coords=[time, lats, lons], dims=['time','lats','lons'])
                    da_landclim = da_empty.assign_attrs(da_landuse)

                    # Compute the product with the "newvalue" column and assign it to a new column in the merged DataFrame

                    # Compute the product with the "newvalue" column and assign it to a new column in the merged DataFrame
                    latitudes = df_sdm['lat'].unique()
                    longitudes = df_sdm['lon'].unique()

                    lats_sorted = np.sort(latitudes)
                    lons_sorted = np.sort(longitudes)

                    # Create a dictionary with (lat, lon) tuples as keys and the corresponding values from df_sdm as values
                    sdm_dict = {(lat, lon): vals for lat, lon, vals in df_sdm[['lat', 'lon', 'vals']].to_numpy()}

                    # Initialize the newvalue_array with NaNs instead of zeros
                    newvalue_array = np.full((len(lats_sorted), len(lons_sorted)), np.nan)

                    # Loop over the latitudes and longitudes and use the dictionary to perform the lookup
                    for i, lat in enumerate(lats_sorted):
                        for j, lon in enumerate(lons_sorted):
                            vals = sdm_dict.get((lat, lon), np.nan)
                            if not np.isnan(vals):
                                newvalue_array[i, j] = vals


                    da = xr.DataArray(newvalue_array, coords=[lats_sorted, lons_sorted], dims=['lat', 'lon'])
                    # Interpolate the values of newvalue to the dimensions of A
                    interpolated_values = da.interp(lat=lats, lon=lons)

                    # Add the interpolated values to the A DataArray
                    da_landuse['newvalue'] = interpolated_values
                    da_landuse['newvalue'] = interpolated_values.fillna(0)
                    
                    
   
                    keys = [row[f"LUH{i}"] for _, row in Habitats_suitable.iterrows() for i in range(1, 5) if pd.notna(row[f"LUH{i}"])]

                    keys = list(set(keys))

                    for variable in list(da_landuse.keys()):
                        if variable not in keys and variable != "newvalue":
                            del da_landuse[variable]

                    num_codes = 0
                    for code in keys: 
                        # Check if the code is "MARINE" and skip land-use filter if it is
                        if code == "MARINE" or not keys:
                            pass
                        else:
                            num_codes += 1
                            # Compute the product with the LUH code and the "newvalue" column, and assign it to a new column in the merged DataFrame
                            np_empty = np.zeros_like(da_landuse[code].values, dtype=float)
                            da_landuse[f"{code}_bin"] = da_landuse[code] * da_landuse["newvalue"]

                            da_landuse[f"{code}_binary"] = (da_landuse[code] > 0).astype(float)
                            da_landuse[f"{code}_lu_binary"] = da_landuse[f"{code}_binary"] * da_landuse["newvalue"]

                            #da_landuse["newvalue_binary"] = (da_landuse["newvalue"] >0).astype(float)
                            #da_landuse[f"{code}_poo_lu_binary"] = da_landuse[f"{code}_binary"] * da_landuse["newvalue_binary"]
                            # Select the DataArrays ending in "_bin"
                            bin_arrays = [da_landuse[var] for var in da_landuse.data_vars if var.endswith("_bin") and var != "sum_bin"]
                            binary_arrays = [da_landuse[var] for var in da_landuse.data_vars if var.endswith("_lu_binary") and var != "sum_lu_binary"]
                            #binary_poo_lu_arrays = [da_landuse[var] for var in da_landuse.data_vars if var.endswith("_poo_lu_binary") and var != "sum_poo_lu_binary"]

                            # Multiply all the arrays together
                            sum_bin = reduce(lambda x, y: x + y, bin_arrays)
                            sum_lu_binary = reduce(lambda x, y: x + y, binary_arrays)
                            #sum_poo_lu_binary = reduce(lambda x, y: x + y, binary_poo_lu_arrays)

                            # Assign the "product_bin" attribute to the da_landuse DataArray
                            da_landuse["sum_bin"] = sum_bin
                            da_landuse["sum_lu_binary"] = sum_lu_binary
                            #da_landuse["sum_poo_lu_binary"] = sum_poo_lu_binary
                            #da_landuse["sum_poo_lu_binary_norm"] = da_landuse["sum_poo_lu_binary"] / num_codes

                    for variable in list(da_landuse.keys()):
                        if variable != "newvalue" and variable != "sum_bin" and variable != "sum_lu_binary":
                            del da_landuse[variable]
                    for code in keys: 
                        # Check if the code is "MARINE" and skip land-use filter if it is
                        if code == "MARINE" or not keys:
                            pass
                        else:


                            output_directory = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model+ "/" + taxa + "/" + model_name + "/"
                            os.makedirs(output_directory, exist_ok=True)


                            da_landuse.to_netcdf("/storage/scratch/users/ch21o450/data/LandClim_Output/" + model+ "/" + taxa + "/" + model_name + "/"  + formatted_species_name + "_" + str(time)+ ".nc")





In [57]:
model

'GBM'

In [43]:
df = pd.DataFrame({"lon": lon, "lat": lat, "vals": z})
df = df.fillna(0)
convcodes_renamed = convcodes.rename(columns={'IUCN_hab':'result.code'})
IUCN['result.code'] = pd.to_numeric(IUCN['result.code'], errors='coerce')
Habitats = IUCN.merge(convcodes_renamed, left_on='result.code', right_on='result.code')

In [45]:
keys = ['LUH1', 'LUH2', 'LUH3', 'LUH4', 'LUH5', 'LUH6', 'LUH7', 'LUH8','LUH9','LUH10', 'LUH11', 'LUH12']
split_cols = Habitats['LUH'].str.split('.', expand=True)
for i, key in enumerate(keys):
    if i < len(split_cols.columns):
        Habitats[key] = split_cols[i]
    else:
        Habitats[key] = pd.Series(dtype='float64')
if len(Habitats.columns) > len(keys) + 1:
    num_missing_cols = len(Habitats.columns) - len(keys) - 1
    Habitats = Habitats.reindex(columns=list(Habitats.columns) + ['LUH{}'.format(i) for i in range(13, 13 + num_missing_cols)], fill_value=np.nan)
    Habitats.drop('LUH', axis=1, inplace=True)
Habitats_suitable = Habitats[Habitats['result.suitability'] == 'Suitable'].copy()

In [47]:
Habitats_suitable.to_markdown()

'|    |   Unnamed: 0 | name                  | region_identifier   |   result.code | result.habitat                                        | result.suitability   |   result.season | result.majorimportance   |   LUH1 |   LUH2 |   LUH3 |   LUH4 |   LUH5 |   LUH6 |   LUH7 |   LUH8 |   LUH9 |   LUH10 |   LUH11 |   LUH12 |   LUH13 |   LUH14 |   LUH15 |   LUH16 |   LUH17 |   LUH18 |   LUH19 |   LUH20 |\n|---:|-------------:|:----------------------|:--------------------|--------------:|:------------------------------------------------------|:---------------------|----------------:|:-------------------------|-------:|-------:|-------:|-------:|-------:|-------:|-------:|-------:|-------:|--------:|--------:|--------:|--------:|--------:|--------:|--------:|--------:|--------:|--------:|--------:|\n|  0 |            1 | Chilonatalus micropus | global              |           7.1 | Caves and Subterranean Habitats (non-aquatic) - Caves | Suitable             |             nan | Yes               

In [21]:

                    
   
                    keys = [row[f"LUH{i}"] for _, row in Habitats_suitable.iterrows() for i in range(1, 5) if pd.notna(row[f"LUH{i}"])]

                    keys = list(set(keys))

                    for variable in list(da_landuse.keys()):
                        if variable not in keys and variable != "newvalue":
                            del da_landuse[variable]

                    num_codes = 0
                    for code in keys: 
                        # Check if the code is "MARINE" and skip land-use filter if it is
                        if code == "MARINE" or not keys:
                            pass
                        else:
                            num_codes += 1
                            # Compute the product with the LUH code and the "newvalue" column, and assign it to a new column in the merged DataFrame
                            np_empty = np.zeros_like(da_landuse[code].values, dtype=float)
                            da_landuse[f"{code}_bin"] = da_landuse[code] * da_landuse["newvalue"]

                            da_landuse[f"{code}_binary"] = (da_landuse[code] > 0).astype(float)
                            da_landuse[f"{code}_lu_binary"] = da_landuse[f"{code}_binary"] * da_landuse["newvalue"]

                            #da_landuse["newvalue_binary"] = (da_landuse["newvalue"] >0).astype(float)
                            #da_landuse[f"{code}_poo_lu_binary"] = da_landuse[f"{code}_binary"] * da_landuse["newvalue_binary"]
                            # Select the DataArrays ending in "_bin"
                            bin_arrays = [da_landuse[var] for var in da_landuse.data_vars if var.endswith("_bin") and var != "sum_bin"]
                            binary_arrays = [da_landuse[var] for var in da_landuse.data_vars if var.endswith("_lu_binary") and var != "sum_lu_binary"]
                            #binary_poo_lu_arrays = [da_landuse[var] for var in da_landuse.data_vars if var.endswith("_poo_lu_binary") and var != "sum_poo_lu_binary"]

                            # Multiply all the arrays together
                            sum_bin = reduce(lambda x, y: x + y, bin_arrays)
                            sum_lu_binary = reduce(lambda x, y: x + y, binary_arrays)
                            #sum_poo_lu_binary = reduce(lambda x, y: x + y, binary_poo_lu_arrays)

                            # Assign the "product_bin" attribute to the da_landuse DataArray
                            da_landuse["sum_bin"] = sum_bin
                            da_landuse["sum_lu_binary"] = sum_lu_binary
                            #da_landuse["sum_poo_lu_binary"] = sum_poo_lu_binary
                            #da_landuse["sum_poo_lu_binary_norm"] = da_landuse["sum_poo_lu_binary"] / num_codes


In [25]:
keys = [row[f"LUH{i}"] for _, row in Habitats_suitable.iterrows() for i in range(1, 5) if pd.notna(row[f"LUH{i}"])]

In [26]:
keys

[]

In [15]:

model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
         '2052', '2056', '2080', '2100', '2150', '2200', '2250']

for taxa in taxas:
    for model in models:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

    species_names = available_names

    def newvalue_fun(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for species_name in species_names:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)

                newvalue = ds["newvalue"]
                sum_bin = ds["sum_bin"]

                newvalue_dict[model_name][species_name] = newvalue
                sum_bin_dict[model_name][species_name] = sum_bin

        projections_dict = {}

        for species_name in species_names:
            value_list = []
            for model_name in model_names:
                value_bin = newvalue_dict[model_name][species_name]
                #value_bin = value_bin.where(value_bin > 0, 1)
                #value_bin = (value_bin > 0.00)

                value_list.append(value_bin)
            value_bin_concat = xr.concat(value_list, dim="model_name")
            mean_value_bin = value_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_value_bin

        value_bin_list = list(projections_dict.values())
        mean_value_bin = xr.concat(value_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_value_bin = mean_value_bin.where(mean_value_bin > 0, 0)
        return mean_value_bin

    def calculate_mean(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}
        lu_sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for species_name in species_names:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)
                sum_bin = ds["sum_bin"]
                #lu_sum_bin = ds["sum_lu_binary"]
                #sum_bin = (sum_bin > 0.00)

                sum_bin_dict[model_name][species_name] = sum_bin
                #lu_sum_bin_dict[model_name][species_name] = lu_sum_bin

        projections_dict = {}

        for species_name in species_names:
            sum_bin_list = []
            for model_name in model_names:
                sum_bin = sum_bin_dict[model_name][species_name]
                sum_bin_list.append(sum_bin)
            sum_bin_concat = xr.concat(sum_bin_list, dim="model_name")
            mean_sum_bin = sum_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_sum_bin

        mean_sum_bin_list = list(projections_dict.values())
        mean_sum_bin = xr.concat(mean_sum_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_sum_bin = mean_sum_bin.where(mean_sum_bin > 0, 0)

        return mean_sum_bin

    historical_time = 1146
    future_times = [35, 65]
    scenario = ["rcp26"]

    netcdf_path_format_future = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_[{}].nc"
    netcdf_path_format_hist = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/EWEMBI/{}_[{}].nc"
    
    future_time=35
    scenario = "rcp26"
    
    mean_value_bin_future = newvalue_fun(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
    mean_sum_bin_future = calculate_mean(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
    mean_value_bin_hist = newvalue_fun(historical_time, model, netcdf_path_format_hist, is_historical=True)
    mean_sum_bin_hist = calculate_mean(historical_time, model, netcdf_path_format_hist, is_historical=True)

    mean_sum_bin_hist = mean_sum_bin_hist.isel(time=0)

    year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

FileNotFoundError: [Errno 2] No such file or directory: b'/storage/scratch/users/ch21o450/data/LandClim_Output/GBM/Mammals/GFDL-ESM2M/rcp26/Chilonatalus_micropus_[35].nc'